In [1]:
import zipfile
import os
from bs4 import BeautifulSoup
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
from collections import defaultdict
import ast
from tqdm.notebook import tqdm as tqdm
import pickle
import contractions

c:\Users\srija\anaconda3\lib\site-packages\scipy\__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


# Bigram Inverted Index

In [2]:
# helper function that generates all bigrams for a given set of tokens and accordingly updates the posting list and the doc frequency corresponding to the bigram.
def helper(inv_idx_freq, inv_idx_postings, list_of_tokens, docID):
    n = len(list_of_tokens)

    for i in range(n-1):
        token = (list_of_tokens[i], list_of_tokens[i+1])
        posting_list =  inv_idx_postings[token]
        if docID in posting_list:
            continue
        else:
            inv_idx_postings[token].append(docID)
            inv_idx_freq[token] = inv_idx_freq[token] + 1
    
    

In [3]:
def t1_and_t2(posting1, posting2):
  
  # if any of the postings is empty then their intersection will also be an empty list
  if len(posting1) == 0 or len(posting2) == 0:
    return []

  ptr1 = 0 #pointer that iterates over posting 1
  ptr2 = 0 #pointer that iterates over posting 2

  comparisons = 0 #intialize number of comparison as 0
  ans = []


  while ptr1 < len(posting1) and ptr2 < len(posting2):
    comparisons += 1 
    if posting1[ptr1] ==  posting2[ptr2]: # this means that both the docs contains the word.
      ans.append(posting1[ptr1])
      ptr1 = ptr1 + 1
      ptr2 = ptr2 + 1

    elif posting1[ptr1] <  posting2[ptr2]:
      ptr1 = ptr1 + 1
    
    else:
      ptr2 = ptr2 + 1

  return ans, comparisons


In [4]:
def not_t1(posting1):
    global docIDs
    ans = [] # to store all the docIDs except that are their in postings1

    for i in docIDs:
        if i in posting1:
            continue
        else:
            ans.append(i)
    return ans
    


In [5]:
def t1_and_not_t2(posting1, posting2):
    ptr1 = 0 #pointer that iterates over posting 1
    ptr2 = 0 #pointer that iterates over posting 2

    comparisons = 0 #intialize number of comparison as 0
    ans = []

    while ptr1 < len(posting1) and ptr2 < len(posting2):
        comparisons += 1

        if posting1[ptr1] == posting2[ptr2]: 
            ptr1+=1
            ptr2+=1
            continue
        elif posting1[ptr1] < posting2[ptr2]:
            ans.append( posting1[ptr1] )
            ptr1+=1

        else:
            ptr2+=1

    while ptr1 < len(posting1):
        ans.append(posting1[ptr1])
        ptr1+=1

    return ans, comparisons

In [6]:
def t1_or_t2(posting1, posting2):

    #if any of the posting list is empty we will return the other list.
    if len(posting1) == 0 :
        return posting2, 0
    if len(posting2) == 0:
        return posting1, 0
        
    ptr1 = 0 #pointer that iterates over posting 1
    ptr2 = 0 #pointer that iterates over posting 2

    comparisons = 0 #intialize number of comparison as 0
    ans = []

    while ptr1 < len(posting1) and ptr2 < len(posting2):
        comparisons+=1
        if posting1[ptr1] < posting2[ptr2]:
            ans.append(posting1[ptr1]) 
            ptr1+=1
        elif posting2[ptr2] < posting1[ptr1]  :
            ans.append(posting2[ptr2])
            ptr2+=1
        else:
            ans.append(posting2[ptr2])
            ptr1+=1
            ptr2+=1

    while(ptr1 < len(posting1)):
        ans.append( posting1[ptr1] )
        ptr1+=1
    
    while(ptr2 < len(posting2)):
        ans.append( posting2[ptr2] )
        ptr2+=1
    
    return ans, comparisons


        

In [7]:
def t1_or_not_t2(posting1, posting2):
    # here we need to take the not of posting2 and then do an or with posting 1

    if len(posting1) == 0: #if the first posting list is empty we will return the not of the other list.
        return not_t1(posting2)
    
    not_posting2 = not_t1(posting2)

    ans, comparisons = t1_or_t2(posting1, not_posting2)
    return ans, comparisons

    
    

In [8]:
inv_idx_freq  = defaultdict(int) # initializing the doc Frequency dictionary as a default dict which return 0 whenver a key does not exists.
inv_idx_postings  = defaultdict(list) # intializing the postings dictionary to return an empty list whenever it is invoked for a key that does not exits.
sample = 2
docIDs = []
folder_path = 'Q1_2_output'
for filename in tqdm(os.listdir(folder_path)):
    file_path = os.path.join(folder_path, filename)
    if os.path.isfile(file_path):
        with open(file_path, 'r') as file:
            contents = file.read()
            docID = int(filename[-4:])
            docIDs.append(docID)
            # print(docID)
            # set_of_tokens = set(contents.strip('][').split(','))
            list_of_tokens = ast.literal_eval(contents)
            helper(inv_idx_freq, inv_idx_postings, list_of_tokens, docID)
            # print(f'{filename}: {contents}')
            # print(set_of_tokens)
            # sample -= 1
            # if (sample == 0):
            #     break


#sort the index in alphabetical order wrt terms
inv_idx_postings = dict(sorted(inv_idx_postings.items()))
inv_idx_freq = dict(sorted(inv_idx_freq.items()))

for key in inv_idx_postings:
    inv_idx_postings[key].sort() #sort the indivdual posting lists correponging to each term in the index


  0%|          | 0/1400 [00:00<?, ?it/s]

In [9]:
# print(inv_idx_postings)

In [10]:
print(inv_idx_postings[('accurate', 'calculation')])

[455]


In [11]:
pickle.dump(inv_idx_postings, open('bi_inv_idx_postings.pkl', 'wb'))
pickle.dump(inv_idx_freq, open('bi_inv_idx_freq.pkl', 'wb'))

In [12]:
inv_idx_postings = pickle.load(open("bi_inv_idx_postings.pkl", 'rb'))
inv_idx_freq = pickle.load(open("bi_inv_idx_freq.pkl", 'rb'))

In [13]:
print(inv_idx_postings[('accurate', 'calculation')])

[455]


# Query Preprocessing

In [14]:
def remove_punc(token):
    translator = str.maketrans('', '', string.punctuation)
    return token.translate(translator)

In [15]:
def preprocess_query(query):
    content1 = query.lower()
    content1 = contractions.fix(content1)
    tokens = word_tokenize(content1)
    stop_words = set(stopwords.words("english"))
    tokens1 = [token for token in tokens if token not in stop_words]
    tokens2 = list(map(remove_punc, tokens1))
    tokens3 = [token for token in tokens2 if token.strip()]
    return tokens3

In [16]:
def bigram(tokens):
    bigram_tokens=[]
    for i in range(0,len(tokens)-1):
        bigram_tokens.append((tokens[i],tokens[i+1]))
    return bigram_tokens

In [17]:
doc_names=os.listdir("Q1_2_output")
queries=int(input())
for i in range(1,queries+1):
    query=input()
    query=preprocess_query(query)
    query=bigram(query)
    q_ptr=1
    answer=None
    if(query[0] in inv_idx_postings.keys()):
        answer=inv_idx_postings[query[0]]
    else:
        answer=[]
    comparisions=0
    while(q_ptr<len(query)):
        T2=query[q_ptr]

        T2_postings=None
        if(T2 in inv_idx_postings.keys()):
            T2_postings=inv_idx_postings[T2]
        else:
            T2_postings=[]

        answer,temp=t1_and_t2(answer, T2_postings)
        comparisions+=temp
        q_ptr+=1

    print(f"Number of documents retrieved for query {i} using bigram inverted index:",len(answer))
    names=", ".join([doc_names[id-1]+".txt" for id in answer])
    print(f"Names of documents retrieved for query {i} using bigram inverted index:",names)
    # print(f"Number of comparisions required for query {i}:",comparisions)

Number of documents retrieved for query 1 using bigram inverted index: 1
Names of documents retrieved for query 1 using bigram inverted index: cranfield0433.txt


# Positional index

In [25]:
def pos_helper(list_of_tokens, docID):
    global term_doc_pos
    global term_doc_freq

    n = len( list_of_tokens )

    for i in range(n):
        token = list_of_tokens[i]
        
        if token in term_doc_pos:
            tdict = term_doc_pos[token]
            
            if docID in tdict:
                tdict[docID].append(i)

            else:
                tdict[docID] = [i]

        else:
            term_doc_pos[token] = { docID : [i] }

        
    

In [26]:
# this function tell wether the given doc contains the given token at the given position
def helper2(docID, token, pos):
    global term_doc_pos
    if token in term_doc_pos:
        tdict =term_doc_pos[token]
        if docID in tdict:
            positions = tdict[docID]
            if pos in positions:
                return True
    return False

# this function tells wether the given docs is valid or not with respect to the position of tokens 
def validDocID(docID, tokens, staring_position):
    n = len(tokens)
    ans = True
    for i in range(n):
        check = helper2(docID, tokens[i], staring_position+i)
        ans =  ans and check

    return ans

#  perform the search of given token using positional index and return the retrieved docIDs
def pos_search(tokens):
    global term_doc_pos
    ans = []
    
    if tokens[0] not in term_doc_pos:
        return []

    # get all the docIDs and the corresponding position indexes, corresponding to this token

    tdict1 = term_doc_pos[tokens[0]]

    for doc_id in tdict1:
        # check if this docID contains all the required token or not at the desired index.
        positions = tdict1[doc_id]
        for j in positions:
            check = validDocID(doc_id, tokens, j)
            if (check):
                ans.append(doc_id)
                break
    return ans
            

    

In [27]:
# term_doc_pos = {
#     'A' : {
#         1 : [0,1,2,3],
#         2 : [0,1,3,4,5,6]
#     },
#     'B' : {
#         1 : [5,6],
#         2 : [7,9]
#     },
#     'C' : {
#         1 : [7,8],
#         2 : [8]
#     }

# }

# pos_search(['B', 'C', 'B'])



In [28]:
# for positional index we create 2 dictionaries.

# the first dictionary has its keys as the token and the values is also a dictionary where the keys are docIDs and the values is a list of positions of the token in that doc id

# the second dictionary has its keys as the token adn the values are the number of docs containing the term.

term_doc_pos = {}
term_doc_freq = {}

sample = 2
docIDs = []
folder_path = 'Q1_2_output'
for filename in tqdm(os.listdir(folder_path)):
    file_path = os.path.join(folder_path, filename)
    if os.path.isfile(file_path):
        with open(file_path, 'r') as file:
            contents = file.read()
            docID = int(filename[-4:])
            docIDs.append(docID)
            list_of_tokens = ast.literal_eval(contents)
            pos_helper(list_of_tokens, docID)
            

for keys in term_doc_pos:
    term_doc_freq[keys] = len( term_doc_pos[keys] )

#sort the index in alphabetical order wrt terms          
term_doc_pos = dict(sorted(term_doc_pos.items()))
term_doc_freq = dict(sorted(term_doc_freq.items()))

# sorting the internal dictionary
for key in term_doc_pos:
    term_doc_pos[key] = dict(sorted(term_doc_pos[key].items()))

    for key2 in term_doc_pos[key]:
        term_doc_pos[key][key2].sort()

  0%|          | 0/1400 [00:00<?, ?it/s]

In [29]:
pickle.dump(term_doc_pos, open('pos_inv_idx_postings.pkl', 'wb'))
pickle.dump(term_doc_freq, open('pos_inv_idx_freq.pkl', 'wb'))

In [30]:
term_doc_pos = pickle.load(open("pos_inv_idx_postings.pkl", 'rb'))
term_doc_freq = pickle.load(open("pos_inv_idx_freq.pkl", 'rb'))

In [31]:
pos_search(['almost', 'cases', 'considered'])

[674]